### 1. check data type and column names

In [ ]:
import pandas as pd

data_path = "data/"

# tweets.dat
print("=== tweets.dat ===")
with open(data_path + "tweets.dat", "r", encoding="utf-8") as f:
    for i in range(5):
        print(f.readline().strip())

# accounts.tsv
print("\n=== accounts.tsv ===")
accounts_df = pd.read_csv(data_path + "accounts.tsv", sep="\t")
print(accounts_df.head())

# media_list.txt
print("\n=== media_list.txt ===")
with open(data_path + "media_list.txt", "r", encoding="utf-8") as f:
    for i in range(5):
        print(f.readline().strip())


### 2. Preprocessing
##### 构造“以图片为粒度”的表，每行是一张图片（media），带上它的 tweet、作者、时间、互动指标、以及账号元数据（Type/Lang/Stance）和文件名。

##### 以author_id 做key, 合并图片 - 推文 - 账号

In [ ]:
import json
from pathlib import Path
from datetime import datetime
import pandas as pd

DATA_DIR = Path("data")


# 1) 读取 accounts.tsv，禁用科学计数，全部按字符串处理
accounts_path = DATA_DIR / "accounts.tsv"
accounts_df = pd.read_csv(
    accounts_path,
    sep="\t",
    dtype=str,           # 防止 author_id 变成 8.50e+06
    keep_default_na=False  # 防止空字符串被当成 NaN
)
# 统一列名
accounts_df.columns = [c.strip() for c in accounts_df.columns]
if "author_id" not in accounts_df.columns:
    # 尝试常见备选列名
    for alt in ["user_id", "id", "account_id"]:
        if alt in accounts_df.columns:
            accounts_df = accounts_df.rename(columns={alt: "author_id"})
            break

print("=== accounts.tsv (head) ===")
display(accounts_df.head())

In [ ]:
# 2) 读 media_list.txt，去掉扩展名，得到 media_key, 存成 media_list_df = [media_key, file_name]
#    例如： "3_456462992792498176.jpg" -> media_key = "3_456462992792498176"
media_list_path = DATA_DIR / "media_list.txt"
media_rows = []
with open(media_list_path, "r", encoding="utf-8") as f:
    for line in f:
        fname = line.strip()
        if not fname:
            continue
        stem = Path(fname).stem  # 去掉扩展名
        media_rows.append({"media_key": stem, "file_name": fname})

media_list_df = pd.DataFrame(media_rows, columns=["media_key", "file_name"])
print("\n=== media_list.txt (head) ===")
display(media_list_df.head())

In [ ]:
# 3) 逐行读取 tweets.dat (JSON Lines)
#    兼容 v2: attachments.media_keys
#         v1: entities.media / extended_entities.media，取 id / id_str 组装成类似 media_key
tweets_path = DATA_DIR / "tweets.dat"

# 以下是G老师写的一些辅助函数, 用于解析推文 JSON, 提取媒体信息, 防止 KeyError 等异常.
def safe_get(d, *keys, default=None):
    """多层 get，避免 KeyError。"""
    cur = d
    for k in keys:
        if not isinstance(cur, dict) or k not in cur:
            return default
        cur = cur[k]
    return cur

def parse_created_at(ts):
    if not ts:
        return None
    # 常见格式："2015-12-12T23:59:59.000Z"
    try:
        return datetime.fromisoformat(ts.replace("Z", "+00:00"))
    except Exception:
        return None

def extract_media_entries(tweet):
    """
    返回 [ {media_key, source} , ... ]
    优先 v2: attachments.media_keys
    如果没有 v2，就去找 v1: entities/extended_entities.media, 这些里通常只有 id/id_str 和 type, 把它拼接成 v2 风格的 media_key = "3_" + id_str, 这样就能和 media_list.txt 对上
    """
    out = []

    # --- v2 路径：attachments.media_keys ---
    media_keys = safe_get(tweet, "attachments", "media_keys", default=[])
    if isinstance(media_keys, list):
        for mk in media_keys:
            out.append({
                "media_key": str(mk),
                "source": "v2_attachments"
            })

    # --- v1 路径：extended_entities.media / entities.media ---
    # 如果没有 v2，就尝试从 v1 里构造 media_key（Twitter v1 通常只有 id/id_str）
    def add_from_media_list(media_list, tag):
        if isinstance(media_list, list):
            for m in media_list:
                mid = str(m.get("id_str") or m.get("id") or "").strip()
                mtype = m.get("type")
                if mid:
                    # 经验上 v2 的 media_key 形如 "3_<id>"，这里用同样格式方便对接 media_list
                    mk = f"3_{mid}"
                    out.append({"media_key": mk,"source": tag})

    ee_media = safe_get(tweet, "extended_entities", "media", default=None)
    if ee_media:
        add_from_media_list(ee_media, "v1_extended_entities")

    e_media = safe_get(tweet, "entities", "media", default=None)
    if e_media:
        add_from_media_list(e_media, "v1_entities")

    # 对同一 tweet 里可能重复收集到的媒体去重（用 media_key）
    unique = {}
    for m in out:
        unique[m["media_key"]] = m
    return list(unique.values())

In [ ]:
# 从 tweet 里还取出了一些基本字段：tweet_id, author_id, created_at, lang, 以及互动指标（retweet_count, reply_count, like_count, quote_count）
# 每张图片一行记录到 image_rows
# 以图片为粒度：后续按天、按账号聚合时，单位是“图片数”“图片收到的互动数”。所以一开始就把“tweet:media = 1:n”拆成每图一行。
image_rows = []

with open(tweets_path, "r", encoding="utf-8") as f:
    for ln, line in enumerate(f, start=1):
        line = line.strip()
        if not line:
            continue
        try:
            tw = json.loads(line)
        except json.JSONDecodeError:
            # 有脏行时跳过
            continue

        tweet_id = str(tw.get("id", "")).strip()
        author_id = str(tw.get("author_id", "")).strip()
        created_at = parse_created_at(tw.get("created_at"))
        lang = tw.get("lang")
        metrics = tw.get("public_metrics") or {}
        retweets = metrics.get("retweet_count")
        replies = metrics.get("reply_count")
        likes = metrics.get("like_count")
        quotes = metrics.get("quote_count")

        media_entries = extract_media_entries(tw)
        if not media_entries:
            continue  # 本推文没有图片

        for m in media_entries:
            image_rows.append({
                "media_key": m["media_key"],
                "tweet_id": tweet_id,
                "author_id": author_id,
                "created_at": created_at,
                "date": created_at.date().isoformat() if created_at else None,
                "lang": lang,
                "retweet_count": retweets,
                "reply_count": replies,
                "like_count": likes,
                "quote_count": quotes,
                "source_path": m["source"]  # 记录提取来源，便于质量检查
            })

images_df = pd.DataFrame(image_rows)

print("\n=== Extracted images from tweets (head) ===")
display(images_df.head())
print(f"Total images extracted: {len(images_df):,}")

In [ ]:
# 4) 关联 media_list（拿到文件名）、accounts（拿到 Type/Lang/Stance）
# 4.1 media_key → file_name
images_df = images_df.merge(media_list_df, how="left", on="media_key")

# 4.2 账号元数据
acc_cols = ["author_id", "Type", "Lang", "Stance"] # 保留lab里要求的列：author_id, Type, Lang, Stance
for c in acc_cols:
    if c not in accounts_df.columns:
        # 容错：如果没这些列，就用空列占位，避免 merge 报错
        accounts_df[c] = ""
# 先在行级别对齐，之后就可以：
#   groupby("date") 做 images_by_day
#   groupby("author_id") 做 images_by_account
images_df = images_df.merge(
    accounts_df[acc_cols].rename(columns={"Lang": "account_lang"}),
    how="left",
    on="author_id"
)

print("\n=== images_df after merge (head) ===")
display(images_df.head())

In [ ]:
# 基础描述性统计，便于写 datasheet
print("\n=== Basic stats ===")
# 总图片数（行数）
print("Total images   :", len(images_df))
# 不同 media_key 数（理论上和总图片数相等，除非有重复记录）
print("Unique media_key:", images_df["media_key"].nunique())
# 不同作者数
print("Unique authors  :", images_df["author_id"].nunique())
# 时间范围
print("Date range      :", images_df["date"].min(), "-", images_df["date"].max())
# 统计发图最多的 5 个账号
print("\nTop 5 accounts by image count:")
display(images_df["author_id"].value_counts().head(5).to_frame("image_count"))
# 发图最多的 5 天
print("\nTop 5 dates by image count:")
display(images_df["date"].value_counts().head(5).to_frame("image_count"))

### 3. 生成后续分析需要的两张表:
##### images_by_day.csv
groupby("date") 统计每一天：图片数量、总 likes/retweets、发图账号数等。
##### images_by_account.csv
groupby("author_id") 统计每个账号：图片数量、互动汇总，附带 Type/stance/account_lang。

In [ ]:
output_dir = Path("output")
output_dir.mkdir(exist_ok=True)

# 1) images_by_day.csv
images_by_day = (
    images_df
    .groupby("date", as_index=False)
    .agg({
        "media_key": "count",             # 每天的图片数量
        "author_id": pd.Series.nunique,   # 发图账号数
        "like_count": "sum",              # 总点赞数
        "retweet_count": "sum"            # 总转推数
    })
    .rename(columns={
        "media_key": "num_images",
        "author_id": "num_accounts",
        "like_count": "total_likes",
        "retweet_count": "total_retweets"
    })
    .sort_values("date")
)

images_by_day_path = output_dir / "images_by_day.csv"
images_by_day.to_csv(images_by_day_path, index=False)
print(f"Saved {images_by_day_path} ({len(images_by_day)} rows)")

# 打印前几行
display(images_by_day.head())


# 2)images_by_account.csv
images_by_account = (
    images_df
    .groupby(["author_id", "Type", "Stance", "account_lang"], as_index=False)
    .agg({
        "media_key": "count",           # 图片数
        "like_count": "sum",
        "retweet_count": "sum",
        "reply_count": "sum",
        "quote_count": "sum"
    })
    .rename(columns={
        "media_key": "num_images",
        "like_count": "total_likes",
        "retweet_count": "total_retweets",
        "reply_count": "total_replies",
        "quote_count": "total_quotes"
    })
    .sort_values("num_images", ascending=False)
)

images_by_account_path = output_dir / "images_by_account.csv"
images_by_account.to_csv(images_by_account_path, index=False)
print(f"Saved {images_by_account_path} ({len(images_by_account)} rows)")

# 打印前几行
display(images_by_account.head())

### 4. 数据质量简报（用于 datasheet）

In [ ]:
print("\n=== Summary ===")
print(f"Total unique images : {images_df['media_key'].nunique():,}")
print(f"Total accounts      : {images_df['author_id'].nunique():,}")
print(f"Date range          : {images_df['date'].min()} → {images_df['date'].max()}")
print(f"Missing file_name   : {images_df['file_name'].isna().sum()}")

# 检查空值比例
missing_ratio = images_df.isna().mean().sort_values(ascending=False).head(10)
print("\nTop 10 columns by missing ratio:")
display(missing_ratio.to_frame("missing_ratio"))
